In [ ]:
import json
from matplotlib.font_manager import json_dump
import numpy as np
import pandas as pd
from pandas import json_normalize
from datetime import datetime
import time
import seaborn as sns
import matplotlib.pylab as plt
import os
  

In [ ]:
# -----------------------------------------------
# Dataset files
# -----------------------------------------------
dataset_path = "/home/gouda/sensorfloor/datasets/debugging_data"
sensorfloor_json_file = "sensorfloor_data.txt"
vicon_json_files = ["vicon_data_person_1.txt", "vicon_data_person_3.txt"]
vicon_object_names = ["person_1", "person_3"]
dataset_type = "train"  # train or test
# -----------------------------------------------

sensorfloor_json_file = os.path.join(dataset_path, sensorfloor_json_file)
vicon_json_files = [os.path.join(dataset_path, vicon_json_file) for vicon_json_file in vicon_json_files]
num_of_vicon_objects = len(vicon_json_files)

In [ ]:
sf_data = []

# txt file is not well formated json, it must be read line by line
for line in open(sensorfloor_json_file, "r"):
    sf_data.append(json.loads(line))

df_sf = pd.DataFrame(sf_data)

## Build the dataframe of sensor floors ##

***Timestamp Interpolation***

In [ ]:
#test timestamp interpolation
#t_batch_i_now = np.zeros((23,15))
t_batch_i_old = np.zeros((23,15))
df_sf_buf = df_sf.copy()


#df_sf_buf = df_sf_buf.reindex(df_sf_buf.columns.tolist() + ['ax','ay','az','gx','gy','gz','mx','my','mz','rssi'], axis=1) 
out = []
for index, row in df_sf_buf.T.items():
    index_strip_id = int(df_sf_buf['column_num'][index])
    index_node_id = int(df_sf_buf['node_id'][index])
    #print('t_batch: ', df_sf_buf['timestamp'][index])
    #t_batch_i_now[index_strip_id][index_node_id] = df_sf_buf['timestamp'][index]
    #print(t_batch_i_old[index_strip_id-1][index_node_id-1])
    delta_t = 0
    timestamp_i = 0

    for i in range(len(df_sf_buf['data'][index])):

        # set previous timestamp
        if t_batch_i_old[index_strip_id-1][index_node_id-1] == 0:  # first sample has no previous timestamp, set it to around 4 seconds before
            delta_t = 4/19
            t_i = (df_sf_buf['timestamp'][index]-4) + ((1+i)*delta_t)
        elif t_batch_i_old[index_strip_id-1][index_node_id-1] < df_sf_buf['timestamp'][index] and t_batch_i_old[index_strip_id-1][index_node_id-1] > 0:
            delta_t = (df_sf_buf['timestamp'][index] - t_batch_i_old[index_strip_id-1][index_node_id-1])/len(df_sf_buf['data'][index])
            t_i = t_batch_i_old[index_strip_id-1][index_node_id-1] + ((1+i)*delta_t)
            
        if df_sf_buf['data'][index][i]['r'][0] < 0:
            rssi_val = df_sf_buf['data'][index][i]['r'][0]
        else:
            rssi_val = np.nan
            #print('iter > 0: ', delta_t)

        out.append({'strip_id':df_sf_buf['column_num'][index],'node_id':df_sf_buf['node_id'][index],
            'ax':df_sf_buf['data'][index][i]['a'][0], 'ay':df_sf_buf['data'][index][i]['a'][1], 'az':df_sf_buf['data'][index][i]['a'][2],
            'gx':df_sf_buf['data'][index][i]['g'][0], 'gy':df_sf_buf['data'][index][i]['g'][1], 'gz':df_sf_buf['data'][index][i]['g'][2],
            'mx':df_sf_buf['data'][index][i]['m'][0], 'my':df_sf_buf['data'][index][i]['m'][1], 'mz':df_sf_buf['data'][index][i]['m'][2],
            'r':rssi_val, 'timestamp':t_i})

        #print(out[index]['strip_id'], out[index]['node_id'],'t_old: ', t_batch_i_old[index_strip_id-1][index_node_id-1], 't_now: ', df_sf_buf['timestamp'][index], out[index]['timestamp'])
        t_first = (df_sf_buf['timestamp'][index]) - 4           
        t_batch_i_old[index_strip_id-1][index_node_id-1] = t_first
        #print(t_batch_i_old[index_strip_id-1][index_node_id-1])
    
    
    t_batch_i_old[index_strip_id-1][index_node_id-1] = df_sf_buf['timestamp'][index]    
    #print(out[index]['strip_id'], out[index]['node_id'],'t_old: ', t_batch_i_old[index_strip_id-1][index_node_id-1], 't_now: ', df_sf_buf['timestamp'][index], out[index]['timestamp'])
        

df_sf_final = pd.DataFrame(out)
del(out)
df_sf_final = df_sf_final.sort_values(['timestamp'])
df_sf_final = df_sf_final.reset_index(drop=True)
df_sf_final

**Build Dataset of VICON Coordinates**

Only the first object of VICON will be loaded here. This first object will be used to synchronize the VICON to the Sensorfloor data.
Other VICON objects will be synchronized to this first object.

In [ ]:
#DATASET OF VICON COORDINATES

def load_vicon_json(vicon_json_file):
    vicon_data = []
    for line in open(vicon_json_file, "r"):
        vicon_data.append(json.loads(line))

    df_vicon = pd.DataFrame(vicon_data)
    #df_vicon['time'] = pd.to_datetime(df_vicon['time'],unit='s')
    df_vicon.shape
    df_vicon_buf = df_vicon.copy()
    df_vicon_buf = df_vicon_buf.reindex(df_vicon_buf.columns.tolist() + ['X','Y'], axis=1) 

    for index, row in df_vicon_buf.T.items():
        for i in range(len(df_vicon_buf['translation'][index])):
            #print((test_df['data'][index][i]['r']))
            df_vicon_buf.loc[index,'X'] = df_vicon_buf['translation'][index][0]
            df_vicon_buf.loc[index,'Y'] = df_vicon_buf['translation'][index][1]

    df_vicon_buf = df_vicon_buf.drop(columns=['object','translation','rotation'])

    return df_vicon_buf

df_vicon_buf = load_vicon_json(vicon_json_files[0])
df_vicon_buf

***Filter and Resample Vicon Data***

Only the first object will be resampled. Other VICON objects will choose closest samples in time to thses samples time stamps of the first VICON object.

In [ ]:
def resample_vicon(df_vicon_buf, resample=False):
    df_vicon_resample = df_vicon_buf.copy()
    df_vicon_resample['time_resample'] = pd.to_datetime(df_vicon_resample['time'],unit='s')
    #df_vicon_resample['time'] = df_vicon_resample['time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f'))
    df_vicon_resample = df_vicon_resample.set_index('time_resample')
    df_vicon_final = df_vicon_resample.resample('40ms').ffill()
    df_vicon_final = df_vicon_final.dropna()
    df_vicon_final = df_vicon_final.sort_values(['time'])
    df_vicon_final = df_vicon_final.reset_index(drop=True)
    return df_vicon_final

df_vicon_final = resample_vicon(df_vicon_buf)
df_vicon_final


**CREATE ARRAY OF RSSI & VICON COORDINATES**

In [ ]:
from turtle import width
import matplotlib.animation as animation
%matplotlib ipympl
from matplotlib.widgets import Slider, Button, RadioButtons

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#CREATE DATASET FOR SENSOR FLOOR ARRAY
x_sf = df_sf_final['strip_id'].to_numpy()
y_sf = df_sf_final['node_id'].to_numpy()
z_sf = df_sf_final['r'].to_numpy()
t_sf = df_sf_final['timestamp'].to_numpy()
dataSet_sf = np.array([x_sf, y_sf, z_sf, t_sf])
numDataPoints_sf = len(t_sf)
num_of_nodes = 15
num_of_strips = 23

delta_t_sf = round((t_sf.max()-t_sf.min())/numDataPoints_sf, 6)

print('dataset sf: ', dataSet_sf.shape)

#-------------------------------------------------------------------------------------------------------------------#

#CREATE DATASET FOR VICON COORDINATES
t_vc = df_vicon_final['time'].to_numpy()
x_vc = df_vicon_final['X'].to_numpy()
y_vc = df_vicon_final['Y'].to_numpy()
dataSet_vc = np.array([x_vc, y_vc, t_vc])
numDataPoints_vc = len(t_vc)

delta_t_vc = round((t_vc.max()-t_vc.min())/numDataPoints_vc, 6)

print("dataset vc:", dataSet_vc.shape)

#-------------------------------------------------------------------------------------------------------------------#
#Check the index of closest timestamp to Vicon Dataset

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    #print(np.abs(array - value), len(np.abs(array - value)))
    return idx


#closest_index_vicon = []
closest_index_sf = []


for i in range(numDataPoints_vc):
    pos = t_vc[i]
    #index_vicon = find_nearest(dataSet_vc[2], pos)
    index_sf = find_nearest(dataSet_sf[3], pos)

    #closest_index_vicon.append(index_vicon)
    closest_index_sf.append(index_sf)

print(len(closest_index_sf))

**CREATE FRAME FOR MERGED DATASET**

In [ ]:
#CREATE ARRAY FOR DATA SENSORS
#from asyncio.windows_events import NULL
from tqdm import tqdm_notebook as tqdm

# path = "D:/Arbeit/6GEM/Program_Development/Sensorfloor/SensorFloor_Evaluation_Tool"
# os.chdir(path)

KEYS = ['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz', 'r', 'timestamp']
Vicon_Coords = pd.read_csv("vicon_node_positions.csv")
df_data = Vicon_Coords[['strip_id','node_id']]
df_data = df_data.astype('int32')

X = np.zeros([len(closest_index_sf), 23, 15, 11])
t = np.zeros([len(closest_index_sf), 1])
frames = []
dataSet_vc_X = []
dataSet_vc_Y = []
dataSet_vc_t = []
cnt = 0
X_i = np.zeros([23, 15, 11])

for index, row in tqdm(df_sf_final.T.items(), total=len(df_sf_final)):

    df_i = row

    for i, key in enumerate(KEYS):
        X_i[int(df_i.strip_id) - 1, int(df_i.node_id) - 1, i] = df_i[key]
    
    X_df = X_i.reshape([345,11])
    df_X = pd.DataFrame(X_df, columns=KEYS)
    df_data[['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz', 'r', 'timestamp']] = df_X
    frame_i = df_data.to_json(orient='columns')
    
    t_i = df_i.timestamp

    # X[index] = X_i
    # t[index] = t_i
    #frames.insert(index, frame_i)
    frame_bool = df_data.empty
    #print(frame_bool)

    if index in closest_index_sf and not frame_bool:
       
        #print(df_data)
        index_assoc = closest_index_sf.index(index)
        #print(cnt, index, index_assoc)
        frames.append(frame_i)
        #frames[cnt] = frame_i
        X[cnt] = X_i
        t[cnt] = t_i
        dataSet_vc_X.append(dataSet_vc[0,index_assoc])
        dataSet_vc_Y.append(dataSet_vc[1,index_assoc])
        dataSet_vc_t.append(dataSet_vc[2,index_assoc])
        #dataSet_vc_final.append(dataSet_vc[:,index_assoc])
        #print(dataSet_vc[0,index_assoc],dataSet_vc[1,index_assoc],dataSet_vc[2,index_assoc])

        # frames[index_assoc] = frame_i
        # X[index_assoc] = X_i
        # t[index_assoc] = t_i
        cnt += 1
    #print(index,'/',len(df_sf_final))

dataSet_vc_final = np.array([dataSet_vc_X, dataSet_vc_Y, dataSet_vc_t])
print(X.shape, t.shape, len(frames), dataSet_vc_final.shape)

**Check index of timestamp threshold**

In [ ]:
sensor_data_final = X[:cnt-1]
t_data_final = t[:cnt-1]
frames_final = frames[:cnt-1]

#dataSet_vc_final = dataSet_vc[:,:]
dataSet_vc_final = dataSet_vc_final[:,:-1]
dataSet_vc_final = np.transpose(dataSet_vc_final)

#print(index_low_X, t[index_low_X] , index_low_vc, dataSet_vc[2][index_low_vc])
print(sensor_data_final.shape, t_data_final.shape, len(frames_final), dataSet_vc_final.shape)

**Read and synchronize other VICON objects**

In [ ]:
# read json files of other vicon objects
other_vicon_obj_filtered_data = []
for i in range(1, num_of_vicon_objects):
    obj_df = load_vicon_json(vicon_json_files[i])
    obj_df = resample_vicon(obj_df, resample=True)

    obj_t_vc = obj_df['time'].to_numpy()
    obj_x_vc = obj_df['X'].to_numpy()
    obj_y_vc = obj_df['Y'].to_numpy()

    # find the closest timestamps in these other objects to the same timestamps of the first vicon object
    closest_index_vicon = []
    for i in range(len(dataSet_vc_final[:,2])):
        pos = dataSet_vc_final[:,2][i]
        index_vicon = find_nearest(obj_t_vc, pos)
        closest_index_vicon.append(index_vicon)

    obj_vicon_data = np.array([obj_x_vc[closest_index_vicon], obj_y_vc[closest_index_vicon], obj_t_vc[closest_index_vicon]])
    
    other_vicon_obj_filtered_data.append(obj_vicon_data)

In [ ]:
np.set_printoptions(suppress=True)

fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(211)
ax1.plot(dataSet_vc[0], dataSet_vc[1], 'r', label='Vicon Object 1')
ax2 = fig.add_subplot(212)
ax2.plot(other_vicon_obj_filtered_data[0][0], other_vicon_obj_filtered_data[0][1], 'r', label='Vicon Object 2')
plt.show()

print(dataSet_vc[0].shape, other_vicon_obj_filtered_data[0][0].shape)

**MERGE RSSI HEATMAP & VICON COORDINATES WITH SLIDER**

In [ ]:
#MERGE RSSI HEATMAP & VICON COORDINATES WITH SLIDER
from turtle import width
import matplotlib.animation as animation
%matplotlib ipympl
from matplotlib.widgets import Slider, Button, RadioButtons
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

rssi_data = sensor_data_final[:,:,:,9]
rssi_data_transpose = np.transpose(rssi_data, [0,2,1])

numdataPoints = len(dataSet_vc_final)

# GET SOME MATPLOTLIB OBJECTS
#fig, (ax1, ax2) = plt.subplots(figsize=(10,12), nrows=2)
fig = plt.figure(figsize=(12,14))
plt.subplots_adjust(bottom=0.25)

# AXES PROPERTIES RSSI HEATMAP
ax1 = fig.add_subplot(311) #(row, column, pos)
#ax1.set_autoscale_on
ax1.set_xlabel('Strip ID')
ax1.set_ylabel('Node ID')
ax1.set_title('RSSI Heatmap')

# AXES PROPERTIES VICON COORDINATES
object_vc_final = other_vicon_obj_filtered_data[0].transpose()
ax2 = fig.add_subplot(312)
ax2.set_xlim(-11.185, 10.185)
ax2.set_ylim(-6.425, 7.575)
ax2.set_xlabel('X(t)')
ax2.set_ylabel('Y(t)')
ax2.set_title('Vicon object: ' + vicon_object_names[0])

# AXES PROPERTIES VICON COORDINATES
ax3 = fig.add_subplot(313)
ax3.set_xlim(-11.185, 10.185)
ax3.set_ylim(-6.425, 7.575)
ax3.set_xlabel('X(t)')
ax3.set_ylabel('Y(t)')
ax3.set_title('Vicon object: ' + vicon_object_names[1])


# Defining the Slider button
# xposition, yposition, width and height
# ax_slide_sf = plt.axes([0.155, 0.02, 0.65, 0.03])
# ax_slide_vc = plt.axes([0.155, 0.07, 0.65, 0.03])
ax_slide_merge = plt.axes([0.155, 0.12, 0.65, 0.03])

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    #print(np.abs(array - value), len(np.abs(array - value)))
    return idx


#DISPLAY INITIAL IMAGE
im_h = ax1.imshow(rssi_data_transpose[numdataPoints-1], cmap="YlGnBu", aspect='auto')
axins = inset_axes(ax1,
                   width="1%",  # width = 5% of parent_bbox width
                   height="90%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 1.25, 1, 1),
                   bbox_transform=ax2.transAxes,
                   borderpad=0,
                   )
cbar = plt.colorbar(im_h, cax=axins, ax=ax1)
cbar.set_label(label='RSSI (dBm)', size=15)
cbar.ax.tick_params(labelsize=12)


#cbar.ax.set_axes_locator
# #cbar.set_ticks()

line1 = ax2.plot(dataSet_vc_final[:,0], dataSet_vc_final[:,1], lw=2, c='g')[0] # For line plot
line2 = ax3.plot(object_vc_final[:,0], object_vc_final[:,1], lw=2, c='g')[0] # For line plot

# Properties of the sliderdf_vicon_final = df_vicon_final.sort_values(['time'])
df_vicon_final = df_vicon_final.reset_index(drop=True)

#normal data
timestamp_merge = Slider(ax_slide_merge, 'Timestamp (s)',
                  t_vc.min(), t_vc.max(), valinit=t_vc.min(), valstep=delta_t_vc)

 

def update_all(val):
    pos = timestamp_merge.val
    #print(pos)
    index = find_nearest(dataSet_vc_final[:,2], pos)
    line1.set_xdata(dataSet_vc_final[:index,0])
    line1.set_ydata(dataSet_vc_final[:index,1])

    index = find_nearest(object_vc_final[:,2], pos)
    line2.set_xdata(object_vc_final[:index,0])
    line2.set_ydata(object_vc_final[:index,1])

    #index2 = find_nearest(dataSet_sf[3], dataSet_vc_final[2][index])
    im_h.set_data(rssi_data_transpose[index])

    #redrawing the figure
    fig.canvas.draw() 

# Calling the function "update" when the value of the slider is changed

timestamp_merge.on_changed(update_all)

#fig.tight_layout()
plt.subplots_adjust(left=0.1, bottom=0.2, right=0.88, top=0.97, hspace=0.2)
#plt.subplots_adjust(left=0.155, bottom=0.2, right=0.91, top=0.97, hspace=0.2)
#plt.subplot_tool()
plt.show()

In [ ]:
plt.savefig(os.path.join(dataset_path, 'visualization.png'), dpi=300)

In [ ]:
#MERGE DATASET

#dataSet_vc_final = np.transpose(dataSet_vc_final)
frame_number = np.empty(dataSet_vc_final.shape[0], dtype=object)
frame_data = np.empty(dataSet_vc_final.shape[0], dtype=object)
frame_count = 0
for row in range(dataSet_vc_final.shape[0]):
    time_vc = dataSet_vc_final[row][2]
    #time_vc = dataSet_vc[2][row]
    #index_vc = closest_timestamp(dataSet_vc[2], time_vc)
    #index_sf = closest_timestamp(t_data_final, time_vc)

    frame_number[row] = str(frame_count)
    frame_data[row] = frames_final[row]
    frame_count += 1

**GENERATES THE DATASET AS .CSV FILE**

In [123]:
other_objects_data = []
column_names = []
if num_of_vicon_objects > 1:
    for obj_num in range(0, num_of_vicon_objects-1):
        obj_filtered_data = other_vicon_obj_filtered_data[obj_num].transpose()
        other_objects_data.append(obj_filtered_data[:,0])
        column_names.append('vicon_' + str(obj_num+1) + '_x')
        other_objects_data.append(obj_filtered_data[:,1])
        column_names.append('vicon_' + str(obj_num+1) + '_y')

if dataset_type == "train":
    array_frame_vc = dataSet_vc_final.astype(object)

    dataSet_final = np.array([frame_number, frame_data, array_frame_vc[:,0], array_frame_vc[:,1]] + other_objects_data)
    dataSet_final = np.transpose(dataSet_final)

    frame_sf = pd.DataFrame(dataSet_final, columns=['frame_number', 'data', 'vicon_0_x','vicon_0_y'] + column_names)

    frame_sf.to_csv(os.path.join(dataset_path, 'train_processed_dataset_objects_' + str(num_of_vicon_objects) + '.csv'), index=False)

# elif dataset_type == "test":
#     array_frame_vc = dataSet_vc_final.astype(object)

#     test_dataset = np.array([frame_number, frame_data])
#     test_dataset = np.transpose(test_dataset)

#     ground_truth = np.array([frame_number, array_frame_vc[:,2], array_frame_vc[:,0], array_frame_vc[:,1]])
#     ground_truth = np.transpose(ground_truth)

#     frame_test = pd.DataFrame(test_dataset, columns=['frame_number','data'])
#     frame_ground_truth = pd.DataFrame(ground_truth, columns=['frame_number','time','vicon_x','vicon_y'])

#     path_dataset = '/media/irfan-flw/DATA/Arbeit/6GEM/Program_Development/6GEM_Dataset/Dataset_Sensor_Floor_Final/24102022/'

#     frame_test.to_csv(os.path.join(dataset_path, 'test_processed_dataset.csv'), index=False)
#     frame_ground_truth.to_csv(os.path.join(dataset_path, 'test_with_gt_processed_dataset.csv'), index=False)

**Store the data to pickle**

In [ ]:
# import pickle

# # write data
# pickle_data = [sensor_data_final, dataSet_vc_final, t_data_final]
# with open(os.path.join(dataset_path, "dataset.pkl"), 'wb') as handle:
#     pickle.dump(pickle_data, handle, protocol=3)
# print('Successfully stored!')